### Executive Summary

The purpose of this analysis is to assess the market opportunity to grow ridership for Lyft Bay Wheels through offers and promotions via their mobile app. 

Upon analysis of the available data, some quality issues have been identified, such as:
* mislabeling of time zone as UTC instead of PST 
* method of data collection for zip code data is subject to errors (e.g. 'nil'); may be better to collect this data based off of credit card information
* Subscriber_type information could be more granular. It's not clear how often subscriber users are churning, which can impact the effectiveness of a promotion strategy.

That said, based on the available data, we are able to determine that ridership behavior is seasonal, where ridership tends to increase during summer months. This may be due to increased tourism and/or an increased desire to enjoy warmer weather by residents of San Francisco. Complementary survey data may help to supplement why seasonal trends occur, particularly among subscribers. 

There are also key locations that are frequented by "commuters." We have defined commuters as any trips that start or end at a train station using the bike station name fields, start_station_name and end_station_name. We've defined train station bike stations as any bike station name containing the words: "caltrain", "rail", or "bart". These are the top stations, which occur within the top 5 most popular trips among commuters: 

1. SF Caltrain 2 (330 Townsend) 
2. Townsend at 7th 
3. Embarcadero at Folsom 
4. SF Caltrain (Townsend at 4th)
5. Harry Bridges Plaza
6. Steuart at Market

Based on these observations, we recommend the following options to be considered as part of the promotional strategy:
* To boost ridership in December, it may be helpful to run a promotion as a holiday special (e.g. give the gift of ridership). 
* A promotion could also be offered in January to capitalize off of new years resolutions (e.g. weight loss, outdoor goals, et cetera). This type of offer may help to convert customers to subscribers. It may also help to stabilize ridership in January. 
* Summer promotions could highlight the impact of bike ridership on the city's carbon footprint, which may persuade residents and tourists to choose bikes vs. ubers/lyfts. 
* Given the high traffic of riders at Harry Bridges Plaza, Lyft may want to host a ridership recruitment event nearby. The event could primarily focus on converting customers to subscribers by highlighting the benefits of bike ridership that current subscribers have already experienced. 
* It might also make sense to offer a promotion to nearby companies to companies in the surrounding areas of the 5 most popular commuter trips. Corporate memberships can be helpful in converting new users and the mobile app could potentially be used to gamify ridership through a point system. Similar to the days of company kickball or softball, employees can potentially compete with neighboring companies for the most miles ridden. 

### Results

#### Part 1 - Querying Data with BigQuery

1. What's the size of this dataset? (i.e., how many trips)

```
SELECT count(*) as row_count
FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
LIMIT 1000
```

```
SELECT count(distinct trip_id) as row_count
FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
LIMIT 1000
```

> Both queries return 983,648.

2. What is the earliest start date and time and latest end date and time for a trip?

```
SELECT min(start_date) as earliest_start_date, max(end_date) as latest_end_date 
FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
LIMIT 1000
```

> * The earliest start date and time for a trip is: 2013-08-29 09:08:00 UTC
> * The latest end date and time for a trip is: 2016-08-31 23:48:00 UTC

3. How many bikes are there?

```
SELECT count(distinct bike_number) as bike_cnt 
FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
LIMIT 1000
```

> There are 700 bikes.

#### Questions of your own

**Question 1:** What is the average trip duration (in minutes)?

 ```
SELECT ROUND(AVG(TIMESTAMP_DIFF(end_date, start_date, MINUTE))) as avg_trip_time
FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
LIMIT 1000
 ```
 
> Answer: 17.0 minutes 

**Question 2:** Which zip code has the most bike trips?

```
SELECT zip_code, count(trip_id) as trip_cnt
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    group by zip_code
    order by trip_cnt desc
    LIMIT 1000 
```

> Answer: 94107

**Question 3:** How many trips started and ended at the same trip station?

```
SELECT count(trip_id) as trip_cnt
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    where start_station_id = end_station_id
    LIMIT 1000
```

> Answer: 32047

#### Part 2 - Querying Data from the BigQuery CLI 

**1. What's the size of this dataset? (i.e., how many trips)**

In [319]:
! bq query --use_legacy_sql=FALSE 'SELECT count(*) as row_count FROM `bigquery-public-data.san_francisco.bikeshare_trips` LIMIT 1000'

#double check with distinct
! bq query --use_legacy_sql=FALSE 'SELECT count(distinct trip_id) as row_count FROM `bigquery-public-data.san_francisco.bikeshare_trips` LIMIT 1000'

Waiting on bqjob_r712e457938690c86_0000017c38f64054_1 ... (0s) Current status: DONE   
+-----------+
| row_count |
+-----------+
|    983648 |
+-----------+
Waiting on bqjob_r52d21953fdb66834_0000017c38f64b83_1 ... (1s) Current status: DONE   
+-----------+
| row_count |
+-----------+
|    983648 |
+-----------+


Both queries return 983,648.

**2. What is the earliest start date and time and latest end date and time for a trip?**

In [320]:
! bq query --use_legacy_sql=FALSE 'SELECT min(start_date) as earliest_start_date, max(end_date) as latest_end_date FROM `bigquery-public-data.san_francisco.bikeshare_trips` LIMIT 1000'

Waiting on bqjob_r68a715303eeb862c_0000017c38f65b08_1 ... (0s) Current status: DONE   
+---------------------+---------------------+
| earliest_start_date |   latest_end_date   |
+---------------------+---------------------+
| 2013-08-29 09:08:00 | 2016-08-31 23:48:00 |
+---------------------+---------------------+


* The earliest start date and time for a trip is: 2013-08-29 09:08:00 UTC
* The latest end date and time for a trip is: 2016-08-31 23:48:00 UTC

**3. How many bikes are there?**

In [321]:
! bq query --use_legacy_sql=FALSE 'SELECT count(distinct bike_number) as bike_cnt FROM `bigquery-public-data.san_francisco.bikeshare_trips` LIMIT 1000'

Waiting on bqjob_raeda813ad6d03a2_0000017c38f66684_1 ... (0s) Current status: DONE   
+----------+
| bike_cnt |
+----------+
|      700 |
+----------+


There are 700 bikes.

**4. How many trips are in the morning vs in the afternoon?**

There are 399,821 trips in the morning and 391,199trips in the afternoon. 

Please note that "morning" and afternoon" were defined as:
* Morning is defined as hours between 6am and 12pm PST.
* Afternoon is defined as hours between 12pm and 6pm PST. 

Based on some exploratory data analysis, we can conclude that start date has been mislabeled. The values in this field denote UTC, but as noted in the schema, the data is actually PST. 

In [ ]:
%%bigquery

select time_of_day, count(distinct trip_id) as trip_cnt from (SELECT
CASE WHEN TIME(start_date) > "05:59:00" and TIME(start_date) < "12:00:00" THEN 'Morning'
WHEN TIME(start_date) > "11:59:00" and TIME(start_date) < "18:00:00" THEN 'Afternoon'
ELSE 'Night' END AS time_of_day, *
FROM `bigquery-public-data.san_francisco.bikeshare_trips` ) t 
WHERE time_of_day = 'Morning' or time_of_day = 'Afternoon'
group by 1

Executing query with job ID: 0471b510-b0f8-4042-b617-82d18bc8388a
Query executing: 0.98s

In [ ]:
%%bigquery hour_data

select EXTRACT(HOUR FROM TIMESTAMP(start_date)) as hour_of_day, count(distinct trip_id) as trip_cnt 
FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
group by 1
order by 1 asc

In [ ]:
#exploratory data analysis: plot hourly data to confirm mislabeling in start date data field

hour_data.plot.bar(x='hour_of_day', rot=0, legend=False)
plt.title("Trips by Hour of the Day")
plt.ylabel("Trip Count")
plt.xlabel("Hour of the Day")

Peak hours of ridership are between 7am and 10am and 4pm and 7pm.

In [ ]:
%%bigquery data

select *
FROM `bigquery-public-data.san_francisco.bikeshare_trips` 

In [ ]:
#exploratory data analysis - inspect data columns for any other data quality issues

num_cols = list(data.columns)
print(num_cols)
len(num_cols)

In [ ]:
#exploratory data analysis - understand count of empty/null values in each column

null_val={}
for col in num_cols:
    null_val[col]=len(data[data[col].isnull()==True])
    print(col,':',null_val[col])

#### Project Questions for Recommendations

In [ ]:
#import libraries for plots 

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.ticker as mtick

**1. What month of the year do customers and subscribers rent bikes the most?**

The most popular month for ridership fluctuates by year. October yielded the most riders in 2014, while July had the most riders in 2015. On average, August and October appear to be the highest months for ridership, but with incomplete data in 2013 and 2016, this isn't very conclusive. 

In [ ]:
%%bigquery data

SELECT format_datetime('%Y', start_date) as year, format_datetime('%m', start_date) as month, count(distinct trip_id) as trip_cnt   
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    GROUP BY 1, 2
    ORDER BY 1 asc 

In [ ]:
#plot a heatmap of bike usage over time

plt.figure(figsize = (25,6))
pt = data.pivot_table(index="year",columns="month",values="trip_cnt", aggfunc="sum").fillna(0)
pt.columns = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
sns.heatmap(pt, annot=True, cmap = 'Blues', alpha = .8, fmt='g')
plt.title("Count of Bike Trips by Month and Year", fontsize =16)
plt.xlabel('Month')

In [ ]:
#plot a horizontal bar chart of bike usage by month

month = data.pivot_table(index= "month",values="trip_cnt", aggfunc="sum").fillna(0)
month.index = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
month.plot.bar(title = "Commuter Ridership by Month")
plt.title("Count of Bike Trips by Month and Year", fontsize =16)
plt.xticks(rotation = 45)
plt.xlabel('Month')
plt.legend(["Trip Count"], loc='upper left', fontsize='large')

**2. How does the trip count differ between customers and subscribers over time?**

In [ ]:
%%bigquery df

SELECT format_datetime('%Y-%m', start_date) as year_month, subscriber_type, count(distinct trip_id) as trip_cnt   
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    GROUP BY 1, 2
    ORDER BY 1 asc 

In [ ]:
#calculate % share of ridership by subscriber type

sub_type = pd.DataFrame(df.groupby(['year_month', 'subscriber_type'])['trip_cnt'].sum()).unstack().fillna(0)
perc = sub_type.div(sub_type.sum(1)/100, axis=0)
ax = perc.plot(kind='bar', stacked=True, title = "Ridership by Month of Year")
ax.set_ylabel("Percent of Trips (%)")
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_xlabel("Month of Year")
plt.legend(["Customer", "Subscriber"], bbox_to_anchor=(1, 1), loc='upper left', fontsize='large')

In [ ]:
%%bigquery cust_data

SELECT format_datetime('%Y', start_date) as year, format_datetime('%m', start_date) as month, count(distinct trip_id) as trip_cnt   
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    WHERE subscriber_type = 'Customer'
    GROUP BY 1, 2
    ORDER BY 1 asc 

In [ ]:
%%bigquery sub_data

SELECT format_datetime('%Y', start_date) as year, format_datetime('%m', start_date) as month, count(distinct trip_id) as trip_cnt   
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    WHERE subscriber_type = 'Subscriber'
    GROUP BY 1, 2
    ORDER BY 1 asc 

In [ ]:
#create subplots of ridership over time by subscriber type

fig, (ax3, ax1, ax2) = plt.subplots(3, 1)
plt.subplots_adjust(hspace=.5) #increase spacing btw plots

#plot trips over time by 'Customers' and 'Subscribers' as a line graph
d = pd.DataFrame(df.groupby(['year_month','subscriber_type'])['trip_cnt'].sum()).unstack()
d.plot(ax = ax3, title = "Trips by Month of Year")
ax3.set_ylabel("Trip Count")
ax3.set_xlabel("Month of Year")
ax3.legend(["Customer", "Subscriber"], bbox_to_anchor=(1, 1), loc='upper left', fontsize='large')

#plot heatmap of 'Customer' ridership by month and year
cd = cust_data.pivot_table(index="year",columns="month",values="trip_cnt", aggfunc="sum").fillna(0)
cd.columns = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
sns.heatmap(cd, ax = ax1, annot=True, cmap = 'Blues', alpha = .8, fmt='g')
ax1.set_title("Customers Bike Trips by Month and Year", fontsize =14)

#plot heatmap of 'Subscriber' ridership by month and year
sd = sub_data.pivot_table(index="year",columns="month",values="trip_cnt", aggfunc="sum").fillna(0)
sd.columns = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
sns.heatmap(sd, ax = ax2, annot=True, cmap = 'Blues', alpha = .8, fmt='g')
ax2.set_title("Subscriber Bike Trips by Month and Year", fontsize =14)

plt.show()

**3. How many trips are starting or ending at a train station?**

**Assumptions:** <br>
* bike stations near train stations are identified by the bike station name
* assume that any bike station name containing the words: "caltrain", "rail", or "bart" are associated with train stations 

In [ ]:
%%bigquery train

SELECT 
CASE WHEN lower(start_station_name) like '%caltrain%' or lower(start_station_name) like '%bart%' or lower(start_station_name) like '%rail%' or
lower(end_station_name) like '%caltrain%' or lower(end_station_name) like '%bart%' or lower(end_station_name) like '%rail%' THEN 'Commuter'
ELSE 'Other' END AS rider_type, count(distinct trip_id) as trip_cnt   
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    GROUP BY 1
    ORDER BY 2 desc 

In [ ]:
train

In [ ]:
#calculate % share of ridership by rider type

train['percent'] = (train['trip_cnt']/train['trip_cnt'].sum())*100
train

**4. Has the share commuter riders increased or decreased over time?**

In [ ]:
%%bigquery ridertype

SELECT format_datetime('%Y-%m', start_date) as year_month, rider_type,  count(distinct trip_id) as trip_cnt FROM
(SELECT CASE WHEN lower(start_station_name) like '%caltrain%' or lower(start_station_name) like '%bart%' or lower(start_station_name) like '%rail%' or
lower(end_station_name) like '%caltrain%' or lower(end_station_name) like '%bart%' or lower(end_station_name) like '%rail%' THEN 'Commuter'
ELSE 'Other' END AS rider_type, *
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` ) x
    GROUP BY 1, 2
    ORDER BY 1 asc

In [ ]:
#plot stacked bar graph 

rider = pd.DataFrame(ridertype.groupby(['year_month', 'rider_type'])['trip_cnt'].sum()).unstack().fillna(0)
percent = rider.div(rider.sum(1)/100, axis=0)
ax = percent.plot(kind='bar', stacked=True, title = "Commuter Ridership by Month of Year")
ax.set_ylabel("Percent of Trips (%)")
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_xlabel("Month of Year")
plt.legend(["Commuter", "Other"], bbox_to_anchor=(1, 1), loc='upper left', fontsize='large')

**5. What are the top 5 most popular trips? Does this differ between 'customers','subscribers', and 'commuters'?**

In [ ]:
#top 5 most popular trips

! bq query --use_legacy_sql=FALSE 'SELECT start_station_name, end_station_name, count(*) as trip_freq FROM `bigquery-public-data.san_francisco.bikeshare_trips` GROUP BY start_station_name, end_station_name ORDER BY trip_freq DESC LIMIT 5'

In [ ]:
%%bigquery dest_cust

SELECT start_station_name, end_station_name, count(distinct trip_id) as trip_cnt   
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    WHERE subscriber_type = 'Customer'
    GROUP BY 1, 2
    ORDER BY 3 desc 
    LIMIT 5

In [ ]:
dest_cust

In [ ]:
%%bigquery dest_sub

SELECT start_station_name, end_station_name, count(distinct trip_id) as trip_cnt   
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` 
    WHERE subscriber_type = 'Subscriber'
    GROUP BY 1, 2
    ORDER BY 3 desc 
    LIMIT 5

In [ ]:
dest_sub

In [ ]:
%%bigquery commuter

SELECT start_station_name, end_station_name, count(distinct trip_id) as trip_cnt FROM
(SELECT CASE WHEN lower(start_station_name) like '%caltrain%' or lower(start_station_name) like '%bart%' or lower(start_station_name) like '%rail%' or
lower(end_station_name) like '%caltrain%' or lower(end_station_name) like '%bart%' or lower(end_station_name) like '%rail%' THEN 'Commuter'
ELSE 'Other' END AS rider_type, *
    FROM `bigquery-public-data.san_francisco.bikeshare_trips` ) x
    WHERE rider_type = 'Commuter'
    GROUP BY 1, 2
    ORDER BY 3 desc
    LIMIT 5

In [ ]:
commuter

### Analysis

Subscribers make up the lion's share of ridership, making up about 80-95% of riders each month with a few exceptions in August and September 2013. That said, low subscribers at the onset makes sense if the ridership program was in its infancy. Both subscribers and customers frequent similar destinations with Harry Bridges Plaza (Ferry Building) and Embarcadero at Sansome reaching the top 5 for as starting stations. 

Based on the bike trip behavior by month and year, there is seasonality in the bike trip behavior. Both customers and subscribers tend to ride less in the winter months. That said, ridership was higher for subscribers in January 2015. Additional context as to why would be helpful in providing recommendations to increase ridership. For example, was it an unusually warm January that year or was there an ad campaign running that promoted more bike usage? 

Overall, commuters, as we've defined it above, make up about 41% of bike ridership, and the share of commuters riding bikes has not changed significantly over time after the first couple of months. Like subscribers and customer segments, commuters also tend to frequent Harry Bridges Plaza (Ferry Building). The top stations among the 5 most popular commuter trips are: 
* SF Caltrain 2 (330 Townsend) 
* Townsend at 7th 
* Embarcadero at Folsom 
* SF Caltrain (Townsend at 4th)
* Harry Bridges Plaza

**Recommendations:** 
* To boost ridership in December, it may be helpful to run a promotion as a holiday special (e.g. give the gift of ridership). 
* A promotion could also be offered in January to capitalize off of new years resolutions (e.g. weight loss, outdoor goals, et cetera). This type of offer may help to convert customers to subscribers. It may also help to stabilize ridership in January. 
* Summer promotions could highlight the impact of bike ridership on the city's carbon footprint, which may persuade residents and tourists to choose bikes vs. ubers/lyfts. 
* Given the high traffic of riders at Harry Bridges Plaza, Lyft may want to host a ridership recruitment event nearby. The event could primarily focus on converting customers to subscribers by highlighting the benefits of bike ridership that current subscribers have already experienced. 
* It might also make sense to offer a promotion to nearby companies to companies in the surrounding areas of the 5 most popular commuter trips. Corporate memberships can be helpful in converting new users and the mobile app could potentially be used to gamify ridership through a point system. Similar to the days of company kickball or softball, employees can potentially compete with neighboring companies for the most miles ridden, fostering company pride and culture.

### Links to the Data 

Link to data can be found by navigating to the following:
   1. Go to BigQuery
   2. Find dataset: bigquery-public-data.san_francisco
   3. Use table: bikeshare_trips

### Descriptions and Schema 

This analysis leverages data from the table, 'bikeshare_trips', which has 983,648 rows in the dataset. It is comprised of 11 columns, which include:

* 'trip_id' : numeric ID of bike trip
* 'duration_sec' : Time of trip in seconds 
* 'start_date' : Start date with date and time in PST
* 'start_station_name' : Station name of start station
* 'start_station_id' : Numeric reference for start station
* 'end_date' : End date of trip with date and time in PST 
* 'end_station_name' : Station name of end station
* 'end_station_id' : Numeric reference for end station
* 'bike_number' : ID of bike used
* 'zip_code' : Home zip code of subscriber. Customers can choose to manually enter zip at kiosk, but data is unreliable.
* 'subscriber_type' : Subscriber = annual or 30-day member; Customer = 24-hour or 3-day member

### Links to Github 

Link to Github repo can be found [here](https://github.com/mids-w205-martin-mims/project-1-edakavlakoglu/tree/assignment).